In [1]:
import pandas as pd
import numpy as np

In [3]:
def get_intents(excel_file):
    patient = pd.read_excel(excel_file, sheet_name=0)
    patient = patient.fillna("-")
    patient['text_all'] = patient.iloc[:,2:].apply(list, axis=1)
    patient_dict = patient.set_index('Informations')['text_all'].to_dict()
    patient_dict_no_nan = {}
    for key, value in patient_dict.items():
        value_no_nan = [x for x in value if x != "-"]
        patient_dict_no_nan[key] = value_no_nan

    sdd = pd.read_excel(excel_file, sheet_name=1)
    sdd = sdd.fillna('-')
    sdd['text_all'] = sdd.iloc[:,2:].apply(list, axis=1)
    sdd_dict = sdd[['Informations', 'type', 'text_all']].to_dict(orient='index')
    sdd_dict_no_nan = {}
    for key, value in sdd_dict.items():
        value_no_nan = [x for x in value['text_all'] if x != "-"]
        if value['Informations'] not in sdd_dict_no_nan.keys():
            sdd_dict_no_nan[value['Informations']] = {value['type']: value_no_nan}
        else:
            sdd_dict_no_nan[value['Informations']][value['type']] = value_no_nan
    sdd_list = []
    for key, value in sdd_dict_no_nan.items():
        tag = {}
        tag['tag'] = key
        tag.update(value)
        sdd_list.append(tag)
    return patient_dict_no_nan, sdd_list

patient_dict_no_nan, sdd_list = get_intents('import/ECOS Intents DC SDD036 I161.xlsx')
patient_dict_no_nan

{'name': ['M. Ledoux'],
 'taille': ['Je fais 1m75'],
 'poids': ['Je fais 92kg'],
 'job': ['Je suis retraité, ancien infirmier. '],
 'motif': ["J'ai mal au dos depuis 4 jours. J'ai vu mon médecin traitant il y a 4 jours il m'a dit que j'avais une colique néphrétique et m'a donné des antalgiques mais là ça ne va pas mieux ! "],
 'stress': ["J'ai mal à mon dos.."],
 'age': ["J'ai 65 ans"],
 'antecedant': ["J'ai déjà eu une fois un crise de goutte pour lequel je prend de l'allopurinol."],
 'antecedantfam': ['Pas à ma connaissance', 'Je ne crois pas'],
 'howareyou': ['Vraiment pas bien Docteur.'],
 'tabac': ['Non Docteur.']}

In [4]:
sdd_list

[{'tag': 'annonce',
  'patterns': ["Je comprends, d'après vos symptomes, vous avez probablement une pyélonéphrite aigue obstructive. Nous allons faire d'autres examens afin de pouvoir confirmer ou exclure cette hypothèse."],
  'responses': ['Vous pouvez me detailler quels examens vous allez me prescrire docteur ? ']},
 {'tag': 'prescriptionExploration',
  'patterns': ['Je vais vous proposer de faire une prise de sang, avec une NFS, une CRP pour recherche une infection, une créatininémie pour evaluer votre fonction rénale. Vous devrez faire un examen urinaire et sanguin pour chercher des bactéries (ECBU et hémocultures). Enfin je vais vous proposer de réaliser un scanner abdominopelvien sans injection pour chercher la cause de vos douleurs et de cette probable pyelonephrite'],
  'responses': ["Attendez Docteur, on m'a déjà fait faire une prise de sang tout à l'heure. Peut etre que vous aurez déjà des resultats ? *** Voici les resultats que vous retrouvez sur le dossier informatisé du pa